In [1]:
# GroupBy技术

import numpy as np
import pandas as pd
from pandas import DataFrame,Series

In [2]:
df = DataFrame({'key1':['a','a','b','b','a'],
               'key2':['one','two','one','two','one'],
               'data1':np.random.randn(5),
               'data2':np.random.randn(5)})
grouped = df['data1'].groupby(df['key1'])# 根据key1的值分组
print(df)
grouped.mean()# 对分组后数字型的列求平均值

      data1     data2 key1 key2
0  1.627136  0.035299    a  one
1  1.578358  1.569246    a  two
2 -0.506083 -1.044514    b  one
3 -0.418653  0.146644    b  two
4  0.759145  0.151646    a  one


key1
a    1.321547
b   -0.462368
Name: data1, dtype: float64

In [4]:
grouped = df[['data1','key1']].groupby(['key1']).agg(np.mean)
grouped

,data1
key1,
a,1.321547
b,-0.462368


In [5]:
# 根据key1/2分组，产生多重索引
means = df['data1'].groupby([df['key1'],df['key2']]).mean()
means

key1  key2
a     one     1.193141
      two     1.578358
b     one    -0.506083
      two    -0.418653
Name: data1, dtype: float64

In [6]:
means.unstack()# 把内层索引变成列

key2,one,two
key1,,
a,1.193141,1.578358
b,-0.506083,-0.418653


In [7]:
states = np.array(['Ohio','California','California',
                  'Ohio','Ohio'])
years = np.array([2005,2005,2006,2005,2006])
df['data1'].groupby([states,years]).mean()
# 使用group后，原始数据可以认为变为如下形式：
#   data1      data2      key1  key2  states      years
# 0 -0.127927   0.026962  a     one   Ohio        2005
# 1 -1.424594  -0.800712  a     two   California  2005
# 2  1.619073  -0.165311  b     one   California  2006
# 3 -0.996192  -0.367086  b     two   Ohio        2005
# 4  0.020317  -1.238209  a     one   Ohio        2006

California  2005    1.578358
            2006   -0.506083
Ohio        2005    0.604242
            2006    0.759145
Name: data1, dtype: float64

In [8]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,1.321547,0.585397
b,-0.462368,-0.448935


In [9]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one   1.193141  0.093473
     two   1.578358  1.569246
b    one  -0.506083 -1.044514
     two  -0.418653  0.146644

In [10]:
# 统计记录条数，类似SQL的group by 然后再count
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [11]:
# 对分组进行迭代
for name,group in df.groupby('key1'):
    print(name)
    print('----')
    print(group)
    print('****')

a
----
      data1     data2 key1 key2
0  1.627136  0.035299    a  one
1  1.578358  1.569246    a  two
4  0.759145  0.151646    a  one
****
b
----
      data1     data2 key1 key2
2 -0.506083 -1.044514    b  one
3 -0.418653  0.146644    b  two
****


In [12]:
for (k1, k2), group in df.groupby(['key1', 'key2']): # 多列分组
    print(k1, k2)
    print('---')
    print(group)
    print('***')

a one
---
      data1     data2 key1 key2
0  1.627136  0.035299    a  one
4  0.759145  0.151646    a  one
***
a two
---
      data1     data2 key1 key2
1  1.578358  1.569246    a  two
***
b one
---
      data1     data2 key1 key2
2 -0.506083 -1.044514    b  one
***
b two
---
      data1     data2 key1 key2
3 -0.418653  0.146644    b  two
***


In [13]:
pieces = dict(list(df.groupby('key1')))
for k,v in pieces.items():
    print(k)
    print('----')
    print(v)
    print('****')

a
----
      data1     data2 key1 key2
0  1.627136  0.035299    a  one
1  1.578358  1.569246    a  two
4  0.759145  0.151646    a  one
****
b
----
      data1     data2 key1 key2
2 -0.506083 -1.044514    b  one
3 -0.418653  0.146644    b  two
****


In [14]:
# 按照每一列的数据类型来分组，
grouped = df.groupby(df.dtypes,axis=1)
for k,v in dict(list(grouped)).items():
    print(k)
    print('----')
    print(v)
    print('****')

float64
----
      data1     data2
0  1.627136  0.035299
1  1.578358  1.569246
2 -0.506083 -1.044514
3 -0.418653  0.146644
4  0.759145  0.151646
****
object
----
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one
****


In [15]:
# 选取某一个或者一组列
print(df.groupby('key1')['data1']) # 等价df['data1'].groupby(df['key1'])
print(df.groupby('key1')[['data2']]) # df[['data2']].groupby(df['key1'])

In [16]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.093473
     two   1.569246
b    one  -1.044514
     two   0.146644

In [17]:
s_grouped = df.groupby(['key1', 'key2'])['data2'] # 这里用'data2'而不是['data2']返回Series
s_grouped.mean()

key1  key2
a     one     0.093473
      two     1.569246
b     one    -1.044514
      two     0.146644
Name: data2, dtype: float64

In [18]:
# 通过字典或者Series进行分组
people = DataFrame(np.random.randn(5,5),
                  columns=['a','b','c','d','e'],
                  index=['Joe','Steve','Wes','Jim','Travis'])
people.loc[2:3,['b','c']] = np.nan #添加空值
people

,a,b,c,d,e
Joe,0.928708,0.394310,1.683041,-0.692119,-0.204378
Steve,-1.541316,-2.581179,1.035769,0.035122,-0.817914
Wes,-0.513045,NaN,NaN,-0.388573,1.751243
Jim,0.172536,-1.991059,-2.217002,-1.207701,-0.162616
Travis,-2.235323,0.140265,0.953341,-0.495427,-0.207430


In [19]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [20]:
by_column = people.groupby(mapping, axis=1) # 每一行根据a/b/c/d/e对应的颜色求sum
by_column.sum()

,blue,red
Joe,0.990921,1.118640
Steve,1.070891,-4.940409
Wes,-0.388573,1.238198
Jim,-3.424703,-1.981139
Travis,0.457914,-2.302487


In [21]:
map_series = Series(mapping)
people.groupby(map_series,axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [22]:
# 通过函数进行分组
# 根据索引名字的长度做group，然后求sum
people.groupby(len).sum()

,a,b,c,d,e
3,0.588199,-1.596749,-0.533961,-2.288394,1.384249
5,-1.541316,-2.581179,1.035769,0.035122,-0.817914
6,-2.235323,0.140265,0.953341,-0.495427,-0.207430


In [23]:
key_list = ['one','one','one','two','two']
people.groupby([len,key_list]).min()
# key_list等于新加一列，结合len，形状如下：
#                  a         b         c         d         e
# Joe(3)     one,  0.254889 -0.812035  2.765460  1.113513  0.646795
# Steve(5)   one,  1.507490  0.463545 -1.396887  0.728163  1.078788
# Wes(3)     one, -2.099479       NaN       NaN  0.438572  0.134136
# Jim(3)     two,  0.167685  1.772127  1.372546 -0.758560 -1.241066
# Travis(6)  two, -0.834662 -0.933228  1.026441 -0.074524 -0.830303

a         b         c         d         e
3 one -0.513045  0.394310  1.683041 -0.692119 -0.204378
  two  0.172536 -1.991059 -2.217002 -1.207701 -0.162616
5 one -1.541316 -2.581179  1.035769  0.035122 -0.817914
6 two -2.235323  0.140265  0.953341 -0.495427 -0.207430

In [25]:
# 根据索引级别来分组
columns = pd.MultiIndex.from_arrays([['US','US',
                                     'US','JP','JP'],
                                     [1,3,5,1,3]],
                                    names=['city','tenor'])
hier_df = DataFrame(np.random.randn(4,5),columns=columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0      1.511381  0.218308  0.702526 -1.205742 -0.103044
1      0.612778  0.068602 -0.080381 -0.881702  0.029807
2      2.456431 -0.597581 -0.358090 -0.027816  0.240721
3     -0.430756  0.904885  0.098086 -1.046797  1.722983

In [26]:
hier_df.groupby(level='city',axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3
